## Corrective RAG

![Corrective RAG](https://blog.langchain.com/content/images/2024/02/data-src-image-811e0f40-3d76-4ab9-9d0d-a720ffdd380b.png)

- [참조 문서](https://blog.langchain.com/agentic-rag-with-langgraph/)
- Travily 를 이용하여 웹 검색을 통한 RAG

In [1]:
import uuid

from IPython.display import HTML
from langchain_core.runnables.graph import Graph


def draw_mermaid_with_html(g: Graph):
    mermaid_code = g.draw_mermaid()

    container_id = f"mermaid-container-{uuid.uuid4().hex[:8]}"

    html = f"""
    <div id="{container_id}"></div>

    <script type="module">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';

        mermaid.initialize({{
            startOnLoad: false,
            theme: 'default',
            securityLevel: 'loose'
        }});

        const graphDefinition = `{mermaid_code}`;
        const container = document.getElementById('{container_id}');

        try {{
            const {{ svg }} = await mermaid.render('graphDiv-' + Date.now(), graphDefinition);
            container.innerHTML = svg;
        }} catch (error) {{
            container.innerHTML = '<pre>' + error + '</pre>';
        }}
    </script>
    """

    display(HTML(html))

## 1. retriever 및 llm 정의

In [2]:
from pathlib import Path

from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings


DATA_DIR = Path("../data")

MODEL_EMBEDDINGS = "bge-m3:567m"
MODEL_LLM = "gpt-oss:120b-cloud"

VECTOR_DB_PERSIST_DIR = DATA_DIR / "chroma_db"
VECTOR_DB_COLLECTION_NAME = "korean_income_tax_law"

embedding_function = OllamaEmbeddings(model=MODEL_EMBEDDINGS)

vector_store = Chroma(
    persist_directory=VECTOR_DB_PERSIST_DIR,
    embedding_function=embedding_function,
    collection_name=VECTOR_DB_COLLECTION_NAME,
)

income_tax_law_retriever = vector_store.as_retriever(kwargs={"k": 1})  # 검색 결과는 1개만

llm = ChatOllama(model=MODEL_LLM)

## 2. Node 및 Conditional Edge 용 함수 만들기

In [3]:
from typing import Literal, TypedDict

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


load_dotenv()

True

In [4]:
from langchain_core.documents import Document


class AgentState(TypedDict):
    query: str
    context: list[Document]
    answer: str

In [5]:
def retrieve(state: AgentState) -> AgentState:
    query = state["query"]
    retrieved_docs = income_tax_law_retriever.invoke(query)
    return {"context": retrieved_docs}

In [6]:
def generate(state: AgentState) -> AgentState:
    query = state["query"]
    raw_context = state["context"]

    prompt = PromptTemplate.from_template("""역할: 당신은 한국의 소득세법 전문가입니다.
---
지시: 오직 주어진 정보를 기반으로 사용자의 질문에 답하세요.
---
정보: {context}
---
사용자 질문: {query}""")
    context = "\n\n".join([doc.page_content for doc in raw_context])
    rag_chain = prompt | llm | StrOutputParser()
    answer = rag_chain.invoke({"query": query, "context": context})
    return {"answer": answer}

In [7]:
llm_for_checking = ChatOllama(model=MODEL_LLM, temperature=0)  # 최대한 같은 답이 나올 수 있도록

In [8]:
def check_doc_relevance(state: AgentState) -> Literal["irrelevant", "relevant"]:
    query = state["query"]
    context = state["context"]
    prompt = PromptTemplate.from_template("""역할: '검색된 문서 내용'과 '사용자의 질문'이 얼마나 연관이 있는지 등급 매기기입니다.
---
지시: 문서가 사용자 질문과 키워드나 의미적으로 관련이 있으면 1, 없으면 0을 반환하세요. 엄격한 기준이 아닌, 명백히 무관한 문서만 필터링하는 것이 목적입니다.
---
출력: 관련이 있으면 'relevant', 없으면 'irrelevant'
---
검색된 문서 내용:
{documents}
---
사용자 질문: {question}""")
    doc_relevance_chain = prompt | llm_for_checking | StrOutputParser()
    return doc_relevance_chain.invoke(
        {
            "question": query,
            "documents": context,
        }
    )

In [9]:
def rewrite(state: AgentState) -> AgentState:
    original_query = state["query"]
    prompt = PromptTemplate.from_template("""역할: 당신은 검색어 변환기입니다.
---
지시: 사용자의 질문을 보고, 웹 검색에 용의하게 변경해주세요.
---
출력: 변경된 질문 내용
---
질문: {query}
""")
    rewrite_query_chain = prompt | llm | StrOutputParser()
    return {"query": rewrite_query_chain.invoke(original_query)}

In [10]:
from langchain_community.retrievers import TavilySearchAPIRetriever


web_search_retriever = TavilySearchAPIRetriever(k=3)


def web_search(state: AgentState) -> AgentState:
    query = state["query"]
    return {"context": web_search_retriever.invoke(query)}

In [11]:
from langgraph.graph import END, START, StateGraph


graph_builder = StateGraph(AgentState)

In [12]:
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate", generate)
graph_builder.add_node("rewrite", rewrite)
graph_builder.add_node("web_search", web_search)

In [13]:
graph_builder.add_edge(START, "retrieve")
graph_builder.add_conditional_edges(
    "retrieve",
    check_doc_relevance,
    {
        "relevant": "generate",
        "irrelevant": "rewrite",
    },
)
graph_builder.add_edge("rewrite", "web_search")
graph_builder.add_edge("web_search", "generate")
graph_builder.add_edge("generate", END)

In [14]:
graph = graph_builder.compile()
draw_mermaid_with_html(graph.get_graph())

In [15]:
graph.invoke({"query": "서울시의 행정 구역 수는?"})

{'query': '서울시 행정구역 수\u200b',
 'context': [Document(metadata={'title': '서울특별시 - 나무위키', 'source': 'https://namu.wiki/w/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C', 'score': 0.84277284, 'images': []}, page_content='서울특별시 ; 하위 행정구역. 25구 ; 면적. 605.2㎢[1] ; 인구. 9,321,863명[2] ; 인구 밀도. 15,405.64명/㎢[3] ; GRDP. $3,841억 (2022)[4].'),
  Document(metadata={'title': '행정구역 - 서울연구데이터서비스', 'source': 'https://data.si.re.kr/data/%EC%A7%80%EB%8F%84%EB%A1%9C-%EB%B3%B8-%EC%84%9C%EC%9A%B8-2013/73', 'score': 0.82994765, 'images': []}, page_content='그 이후에도 행정동의 설치, 폐지로 인해 2010년 기준 서울시의 행정구역은 25개 자치구와 424개의 행정동이 되었다. 행정동은 행정의 편의를 위해 설정한 행정구역으로서'),
  Document(metadata={'title': '서울특별시 - 위키백과, 우리 모두의 백과사전', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C', 'score': 0.7950386, 'images': []}, page_content='서울특별시의 행정구역은 2022년 5월말 기준으로 25개 자치구와 426개 행정동이 ... 특히, 최신 트렌드를 반영한 관광 정보를 발 빠르게 제공하여 관광객들이 서울을 제대로')],
 'answer': '서울특별시의 행정구역은 다음과 같습니다.\n\n- **자치구(구) 수:** 25개  \n- **행정